Will Rice\
COMP 5600\
Hw03\
September 22, 2020


## Report on 1. Hill Climbing
Below is the notebook that I used for hw03. I have a seperate file for the QueenBoard. Here is my reporting on the required information:

My definition of the score function: fitness = 0 - numberOfTotalConflicts. The numberOfTotalConflicts was found by looking at each queen and determining how many other queens they ran into. This lead to each conflict being counted twice by design. The best possible score would be zero.

How I generated neigbors: I looked at the number of conflicts that each queen had and chose the one with the most conflicts. I tie broke by choosing the queen on the highest row.

The best state that I could find is below:

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]\
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]\
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]\
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]\
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
 
 There are 4 queens in conflict in the best result above.
 
 The approach I used was WALKSAT. Basically, three random neighbors were chosen. Then if there were improvements, then it immeadiatly took those improvements. If there were no improvements, then 50% of the time it took the least bad neighbor of those chosen, and the other 50% of the time it took a random neighbor of the three chosen.
 
 The WALKSAT approach performed much better than the vanilla hill climbing algorithm because it did not easily get stuck in local optimas.

## Report on 2. Simulated Annealing
Further down in the notebook is my implementation of simulated annealing. Here is my report on it below:

In my testing, I found that a higher starting temperature with a somewhat large alpha yielded the best results. This approach lead to more random results being chosen, which got the problem out of local optimas.

On average, I would say that the Simulated Annealing is tied with teh WALKSAT for best algorithm that I tried for this problem. Both can consistently get the score to be greater than -14. More tuning and a better neighborhood structure could probably get Simulated Annealing to be the overall best though.

In [1]:
import numpy as np
import random
import operator
import math
from src.board import QueenBoard

In [2]:
board = QueenBoard(createNew=True, board=[])

In [3]:
def pickRow(board):
    pickedRow = -1
    worstFit = -1
    for row in board.conflictDict:
        #print(f"{row}: {board.conflictDict[row]}")
        if board.conflictDict[row] > worstFit:
            worstFit = board.conflictDict[row]
            pickedRow = row

    return pickedRow

In [4]:
def getVanillaNeighbors(board):
    pickedRow = pickRow(board)
    freshBoard = board.board.copy()
    for i in range(25):
        freshBoard[pickedRow][i] = 0
    
    neighbors = []
    for i in range(25):
        cur = freshBoard.copy()
        cur[pickedRow][i] = 1
        neighbors.append(QueenBoard(createNew=False, board=cur))
    
    return neighbors
        

In [5]:
# Vanilla Hill Climb

board = QueenBoard(createNew=True, board=[])
i = 0
bestFitVanilla = -10000
bestBoardVanilla = board

while i < 1000 and bestFitVanilla != 0:
    i += 1
    neighbors = getVanillaNeighbors(board)
    neighbors.sort(key=operator.attrgetter('fitness'))
    board = neighbors[-1]
    if board.fitness > bestFitVanilla:
        bestFitVanilla = board.fitness
        bestBoardVanilla = board

print(bestBoardVanilla.board)
print(bestBoardVanilla.fitness)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [6]:
# Variation 4 of hill climbing: WALKSAT

board = QueenBoard(createNew=True, board=[])
i = 0
bestFitWalksat = -10000
bestBoardWalksat = board

while i < 1000 and bestFitWalksat != 0:
    i += 1
    curFit = board.fitness
    neighbors = getVanillaNeighbors(board)
    
    neighbors = random.sample(neighbors, k=3)
    neighbors.sort(key=operator.attrgetter('fitness'))
    bestNeighbor = neighbors[-1]
    
    if bestNeighbor.fitness > curFit:
        board = bestNeighbor
    else:
        useBest = random.randint(0, 1)
        if useBest == 0:
            board = random.choice(neighbors)
        else:
            board = bestNeighbor
        
    
    if board.fitness > bestFitWalksat:
        bestFitWalksat = board.fitness
        bestBoardWalksat = board

print(bestBoardWalksat.board)
print(bestBoardWalksat.fitness)
print(bestBoardWalksat.conflictDict)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [7]:
# Simulated Annealing Schedule
def schedule(prevT):
    return prevT * 0.95

In [8]:
# Simulated Annealing

board = QueenBoard(createNew=True, board=[])
i = 0
bestFitSA = -10000
bestBoardSA = board

T = 1000
while i < 1000:
    i += 1
    curFit = board.fitness
    neighbors = getVanillaNeighbors(board)
    
    T = schedule(T)
    if T == 0:
        break
    nextBoard = random.choice(neighbors)
    deltaE = nextBoard.fitness - curFit
    
    if deltaE > 0:
        board = nextBoard
    else:
        prob = math.exp(deltaE / T)
        randProb = random.uniform(0,1)
        if randProb < prob:
            board = nextBoard
    
    if board.fitness > bestFitSA:
        bestFitSA = board.fitness
        bestBoardSA = board

print(bestBoardSA.board)
print(bestBoardSA.fitness)
print(bestBoardSA.conflictDict)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 